<a href="https://colab.research.google.com/github/Sakib635/sage2.0/blob/main/5th_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
!pip install z3-solver
from z3 import Solver, Bool, Or, And, Implies, sat, Int, String, Not, Real, simplify, Optimize


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 8.4 MB/s eta 0:00:00


In [97]:
import os
import json
import re
import time
import logging

In [112]:
# Function to read the requirements.txt file from a directory
def read_requirements(directory):
    with open(os.path.join(directory, 'r3.txt'), 'r') as file:
        return file.read()
# Function to read the JSON file from a directory
def read_json_file(directory, filename='updated_formated_8k.json'):
    with open(os.path.join(directory, filename), 'r') as file:
        return json.load(file)

In [113]:
def parse_requirements(requirements_txt):
    requirements = {}
    lines = requirements_txt.strip().split('\n')
    for line in lines:
        line = line.strip()
        if line:
            parts = re.split(r'([><!=]=?[\d.*]+(?:, )?)', line)
            package = parts[0].strip()
            version_specs = parts[1:]

            for spec in version_specs:
                if spec.strip():
                    match = re.match(r'([><!=]=?)([\d.*]+)', spec.strip())
                    if match:
                        operator, version = match.groups()
                        if package in requirements:
                            requirements[package].append((operator, version))
                        else:
                            requirements[package] = [(operator, version)]
    return requirements

In [114]:
def version_satisfies(version, spec):
    """
    Check if a version satisfies a given version specifier.
    """
    operator, spec_version = spec
    if operator == '==':
        return re.match(spec_version.replace('*', '.*'), version) is not None
    elif operator == '!=':
        return re.match(spec_version.replace('*', '.*'), version) is None
    elif operator == '>':
        return version > spec_version
    elif operator == '>=':
        return version >= spec_version
    elif operator == '<':
        return version < spec_version
    elif operator == '<=':
        return version <= spec_version
    return False

def find_matching_versions(package, specs, projects_data):
    """
    Find all versions of a package that satisfy the given version specifiers.
    """
    if package not in projects_data:
        return []

    versions = projects_data[package].keys()
    matching_versions = []
    for version in versions:
        if all(version_satisfies(version, spec) for spec in specs):
            matching_versions.append(version)
    return matching_versions

def fetch_direct_dependencies(requirements, projects_data):
    """
    Fetch direct dependencies for each package based on the parsed requirements.
    """
    direct_dependencies = {}

    for package, specs in requirements.items():
        matching_versions = find_matching_versions(package, specs, projects_data["projects"])
        direct_dependencies[package] = matching_versions

    return direct_dependencies

In [115]:
def parse_dependency(dependency):
    """
    Parse a dependency string into a package and a list of version specifiers.
    """
    parts = re.split(r'([><!=]=?[\d.*]+(?:, )?)', dependency)
    package = parts[0].strip()
    version_specs = []

    for spec in parts[1:]:
        if spec.strip():
            match = re.match(r'([><!=]=?)([\d.*]+)', spec.strip())
            if match:
                operator, version = match.groups()
                version_specs.append((operator, version))

    return package, version_specs

def fetch_transitive_dependencies(direct_dependencies, projects_data):
    """
    Recursively fetch transitive dependencies for each version of the packages in direct dependencies.
    """
    transitive_dependencies = {}

    def _fetch(package, version):
        key = f"{package}=={version}"
        if key in transitive_dependencies:
            return transitive_dependencies[key]

        version_data = projects_data["projects"][package][version]
        dependencies = {}
        if version_data["dependency_packages"]:
            for dep in version_data["dependency_packages"]:
                dep_package, dep_specs = parse_dependency(dep)
                matching_versions = find_matching_versions(dep_package, dep_specs, projects_data["projects"])
                dependencies[dep_package] = matching_versions
                for dep_version in matching_versions:
                    _fetch(dep_package, dep_version)

        transitive_dependencies[key] = dependencies

    for package, versions in direct_dependencies.items():
        for version in versions:
            _fetch(package, version)

    return transitive_dependencies

In [116]:
def generate_smt_expression(direct_dependencies, transitive_dependencies):
    # Initialize an Optimize solver to handle both hard and soft constraints
    solver = Optimize()
    constraints = []

    # Generate constraints for direct dependencies
    for package, versions in direct_dependencies.items():
        if isinstance(versions, list):
            # Create a constraint that the package version must be one of the specified versions
            package_constraint = Or([String(package) == v for v in versions])
            constraints.append(package_constraint)
            # Add soft constraints with weights for versions
            sorted_versions = sorted(versions, reverse=False)  # Sort versions to prioritize newer versions
            weight = 1
            for version in sorted_versions:
                # Add a soft constraint with increasing weight for newer versions
                solver.add_soft(String(package) == version, weight)
                weight += 1  # Increment the weight for the next version

    # Generate constraints for transitive dependencies
    for package_version, dependencies in transitive_dependencies.items():
        if isinstance(dependencies, dict):
            # Split the package_version to get the package name and its version
            package, version = package_version.split('==')
            for dep_package, dep_versions in dependencies.items():
                # Create a constraint for each dependency that it must be one of the specified versions
                dependency_constraint = Or([String(dep_package) == dep_version for dep_version in dep_versions])
                constraints.append(Implies(String(package) == version, dependency_constraint))
                # Add soft constraints with weights for versions
                sorted_versions = sorted(dep_versions, reverse=False)  # Sort versions to prioritize newer versions
                weight = 1
                for dep_version in sorted_versions:
                    # Add a soft constraint with increasing weight for newer versions
                    solver.add_soft(String(dep_package) == dep_version, weight)
                    weight += 1  # Increment the weight for the next version

    # Combine all constraints into a single final constraint
    final_constraint = And(constraints)
    solver.add(final_constraint)

    return solver, constraints

In [117]:
# Function to solve the SMT expression
def smt_solver(solver):
    # Check for the maximum satisfaction
    if solver.check() == sat:
        # Get the model
        model = solver.model()
        return {d.name(): model[d] for d in model.decls()}
    else:
        print("Not satisfiable.")
        return None

In [120]:
def main():
    directory = '/content/drive/MyDrive/smart pip sample data'



    # Log file setup
    log_file = 'execution_log.txt'

    def log_execution_time(action_name, start_time, end_time):
        with open(log_file, 'a') as file:
            file.write(f'{action_name} execution time: {end_time - start_time} seconds\n')


    # Read files from the directory
    start_time = time.time()
    requirements_txt = read_requirements(directory)
    projects_data = read_json_file(directory)
    end_time = time.time()
    log_execution_time("Reading files", start_time, end_time)



    # Parse requirements
    start_time = time.time()
    requirements = parse_requirements(requirements_txt)
    # assert parse_requirements("python-sat>=3.1") == ("python-sat", [(">=", 3.1)])
    end_time = time.time()
    log_execution_time("Parsing requirements", start_time, end_time)
    print("Parsed requirements:", requirements)


    # Fetch matching versions and their dependencies
    start_time = time.time()
    direct_dependencies = fetch_direct_dependencies(requirements, projects_data)
    end_time = time.time()
    log_execution_time("Fetching versions and dependencies", start_time, end_time)
    print("Direct dependencies:", direct_dependencies)


    # Fetch matching versions and their dependencies
    start_time = time.time()
    transitive_dependencies = fetch_transitive_dependencies(direct_dependencies, projects_data)
    end_time = time.time()
    log_execution_time("Fetching transitive dependencies dependencies", start_time, end_time)
    print("transitive dependencies  :", transitive_dependencies)

    # Generate SMT expression
    start_time = time.time()
    solver, constraints = generate_smt_expression(direct_dependencies, transitive_dependencies)
    end_time = time.time()
    log_execution_time("Generating SMT expression", start_time, end_time)
    # Save to a file (optional)
    with open('SMT_expression.txt', 'w') as file:
      file.write(str(solver))


    # Solve the SMT expression
    start_time = time.time()
    solution = smt_solver(solver)
    end_time = time.time()
    log_execution_time("Solving SMT expression", start_time, end_time)
    print(f'Optimal Solution: {solution}')

if __name__ == "__main__":
    main()

Parsed requirements: {'idna': [('>', '2.6')], 'requests': [('==', '2.22.0')]}
Direct dependencies: {'idna': ['3.7', '3.6', '3.5', '3.4', '3.3', '3.2', '3.1', '3.0', '2.9', '2.8', '2.7'], 'requests': ['2.22.0']}
transitive dependencies  : {'idna==3.7': {}, 'idna==3.6': {}, 'idna==3.5': {}, 'idna==3.4': {}, 'idna==3.3': {}, 'idna==3.2': {}, 'idna==3.1': {}, 'idna==3.0': {}, 'idna==2.9': {}, 'idna==2.8': {}, 'idna==2.7': {}, 'chardet==3.0.4': {}, 'chardet==3.0.3': {}, 'chardet==3.0.2': {}, 'idna==2.6': {}, 'idna==2.5': {}, 'urllib3==1.25.9': {}, 'urllib3==1.25.8': {}, 'urllib3==1.25.7': {}, 'urllib3==1.25.6': {}, 'urllib3==1.25.5': {}, 'urllib3==1.25.4': {}, 'urllib3==1.25.3': {}, 'urllib3==1.25.2': {}, 'urllib3==1.25': {}, 'urllib3==1.24.3': {}, 'urllib3==1.24.2': {}, 'urllib3==1.24.1': {}, 'urllib3==1.24': {}, 'urllib3==1.23': {}, 'urllib3==1.22': {}, 'urllib3==1.21.1': {}, 'certifi==2024.2.2': {}, 'certifi==2023.11.17': {}, 'certifi==2023.7.22': {}, 'certifi==2023.5.7': {}, 'certifi==2